In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
city = '대구'

path = 'Data/Variants/Infected/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files[city]
data

,stdDay,incDec,alpha_variants,delta_variants
0,2021-04-12,16.0,4.16,0.32
1,2021-04-13,9.0,2.34,0.18
2,2021-04-14,11.0,2.86,0.22
3,2021-04-15,18.0,4.68,0.36
4,2021-04-16,4.0,1.04,0.08
...,...,...,...,...
121,2021-08-11,66.0,0.66,62.70
122,2021-08-12,39.0,0.39,37.05
123,2021-08-13,57.0,0.57,54.15
124,2021-08-14,82.0,0.82,77.90


In [3]:
learning_rates_list = [1e-3, 1e-4]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]
criterions_list = [nn.MSELoss(), criterion3]
batch_size = 64
hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for hidden_size in hidden_sizes_list:    
                for dropout in dropout_list:
                    for criterion in criterions_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, hidden_size, dropout, criterion]
                        i += 1

print(i)

72


In [ ]:
trained_data = {}
city_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', 
             '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
               'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']


for city in tqdm(city_list, desc = 'cities', position = 0):
    data = dic_files[city]
    best_model = {}
    
    input_size = 3
    sequence_length = 60
    num_epochs = 10000

    df = Prepare_df.processing(data, 'stdDay', 'alpha_variants')

    x = df.iloc[:, 0:]
    y = df.iloc[:,:1]

    ms = MinMaxScaler()
    ss = StandardScaler()

    ss.fit(x)
    ms.fit(y)

    for num_model in tqdm(range(12), desc = f'{city}', position = 1):
        rmse_min = np.inf
        h = -1
        
        for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'{city}_{models_list[num_model]}', position = 2):
            h += 1
            if (num_model < 6) and (h % 2 == 1):
                continue
            model_name = models_list[num_model]
            lr = hyperparameter[0]
            patience = hyperparameter[1]
            num_layers = hyperparameter[2]
            hidden_size = hyperparameter[3]
            dropout = hyperparameter[4]

            if num_model < 6:
                criterion = nn.MSELoss()
            else:
                criterion = hyperparameter[5]

            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 57, 60, 1, batch_size, 'mto')
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 51, 60, 7, batch_size, 'mtm')

            if num_model == 0:
                model = RNN(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 1:
                model = LSTM(input_size = input_size,
                             hidden_size = hidden_size,
                             sequence_length = sequence_length,
                             num_layers = num_layers, 
                             dropout = dropout, 
                             device = device).to(device)

            elif num_model == 2:
                model = GRU(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 3:
                model = BiRNN(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 4:
                model = BiLSTM(input_size = input_size,
                               hidden_size = hidden_size,
                               sequence_length = sequence_length,
                               num_layers = num_layers, 
                               dropout = dropout, 
                               device = device).to(device)

            elif num_model == 5:
                model = BiGRU(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 6:
                model = RNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 7:
                model = LSTM_encoder_decoder(input_size = input_size, 
                                             hidden_size = hidden_size,
                                             num_layers = num_layers, 
                                             dropout = dropout,
                                             device = device).to(device)

            elif num_model == 8:
                model = GRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 9:
                model = BiRNN_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)

            elif num_model == 10:
                model = BiLSTM_encoder_decoder(input_size = input_size, 
                                                  hidden_size = hidden_size,
                                                  num_layers = num_layers, 
                                                  dropout = dropout,
                                                  device = device).to(device)

            elif num_model == 11:
                model = BiGRU_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)            

            optimizer = Adam(model.parameters(), lr = lr)

            if num_model < 6:
                loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                              test_loader, 
                                                              model, 
                                                              criterion, 
                                                              optimizer, 
                                                              num_epochs, 
                                                              patience,
                                                              device)

                label_y, predicted = predict_mto(model, df, x_ss, y_ms)
                
                if rmse_min > rmse(label_y[-7:], predicted[-7:]):
                    rmse_min = rmse(label_y[-7:], predicted[-7:])
                    best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                          num_layers,hidden_size, dropout]
                
                
            else:
                loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                               test_loader, 
                                                               model, 
                                                               criterion, 
                                                               optimizer, 
                                                               num_epochs, 
                                                               patience,
                                                               7,
                                                               device)        
                
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                                                7, 0.5, device)
    
                if rmse_min > rmse(label_y[-7:], predicted[-7:]):
                    rmse_min = rmse(label_y[-7:], predicted[-7:])
                    best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                          num_layers, hidden_size, dropout, criterion]
            
        
            
    trained_data[city] = best_model

cities:   0%|          | 0/17 [00:00<?, ?it/s]

강원:   0%|          | 0/12 [00:00<?, ?it/s]

강원_RNN:   0%|          | 0/72 [00:00<?, ?it/s]

강원_LSTM:   0%|          | 0/72 [00:00<?, ?it/s]

강원_GRU:   0%|          | 0/72 [00:00<?, ?it/s]

강원_BiRNN:   0%|          | 0/72 [00:00<?, ?it/s]

강원_BiLSTM:   0%|          | 0/72 [00:00<?, ?it/s]

강원_BiGRU:   0%|          | 0/72 [00:00<?, ?it/s]

In [ ]:
trained_cities

In [ ]:
#for city in trained_cities:
#    for name, model in trained_cities[city].items():
#        save_model(model.state_dict(), f"model/Infected/{city}/alpha/{name}.pth")